In [ ]:
#Approach:
# Load ground states into a Class.
# Step through all states (levels) loading isospin
# Loop through stable states
#   Get excitations with M1, get M1, isospin, energy
# Loop through unstable states
#   Find decays that can produce stable states with excitations we care about
#   Get log fts
# Step through M1s
#   Calculate isospin rotation coefficient
#     Calculate T3
#   Calculate BGT0
#   Store in structure
# Plot
import numpy as np
import utils

# 1. Get ground states
# Class to load Z,N,Symbol,JP,isospin,mass,mass unc
ground_states = utils.parseGroundStates("data/all.csv")

# 2. Get M1 levels, isospins
for i,j in np.ndindex(ground_states.shape):
  if not ground_states[i][j]==0:
    A = int(ground_states[i][j].Z + ground_states[i][j].N)
    symbol = str(A)+ground_states[i][j].symbol
    print(symbol)
    levels = utils.parseGammas(symbol)

# 3. Get associated beta decays with each M1

# 4. Step through M1 transitions, calculating rotation coefficent, BGT0. Add to level

# 5. Plot


1H




2H




3He




4He



















6Li






















7Li
















9Be



























































10B

























































































11B








































































12C

























































































































13C
















































































































14N

































































































































































































15N











































































































































































































































